# convertBCODMOtoCMAP
Krista Longnecker, 2 July 2025\
Run this after running getBCODMOinfo.ipynb\
Once you have the information from BCO-DMO dataset in hand, use this to make it into the format required by CMAP.\
Note: this version puts most of the code into a separate python script (convert.py) to make it easier to run through multiple data files.

In [30]:
#some of these are residual from assembling the data file, keep for now.
import pandas as pd
import requests
import os
import json
import re
import pdb

from datetime import datetime, timedelta, timezone
from SPARQLWrapper import SPARQLWrapper, POST, JSON
from frictionless import describe, Package

In [2]:
# Make a function that searches for bcodmo:name and returns bcodmo:description and bcodmo:units
# input: md --> the list of parameters for one dataset
def getDetails(md,bcodmo_name):
    """
    Take the list of information from BCO-DMO, search for a name, and return the description and units for that name
    """
    for i, item in enumerate(md):
        if item['bcodmo:name'] == bcodmo_name:
            #actually want the descrption, so return that
            description = item['bcodmo:description']
            units = item['bcodmo:units']

    return description, units

In [3]:
#read in the package that was already made (using getBCODMOinfo.ipynb)
biosscope = Package('datapackage.json')
# biosscope = Package(JSON_FILE)

In [19]:
out = [];
for idx,item in enumerate(biosscope.resources):
    justFile = item.name;
    #pdb.set_trace()
    out.append(justFile)

out #this will be a list, iterate through the files on this list 
#leave here so I see the filenames might not need the actual list

['zooscan_images_bats_ae1614',
 'zooscan_images_bats_ae1712',
 'zooscan_images_bats_ae1819',
 'zooscan_images_bats_ae1830',
 'zooscan_bats_biovolume',
 'zooscan_mocness_output',
 'survey_biogeochemical',
 '920443_v1_biosscope_in_situ_pump_chemical_data',
 '964684_v1_amino_acids_biosscope_2021',
 '964801_v1_pump_carbohydrates_biosscope_2021',
 '964826_v1_pump_poc_pon_biosscope']

In [40]:
#most datasets need additional processing to get depth and/or time. 
#Set up a table here to note what needs to be done, use later to spin off to other scripts.
toSkip = {'zooscan_images_bats_ae1614', #these actually will not match below bc do not end in csv
 'zooscan_images_bats_ae1712',
 'zooscan_images_bats_ae1819',
 'zooscan_images_bats_ae1830'}
addDepth = {'920443_v1_biosscope_in_situ_pump_chemical_data',
 '964684_v1_amino_acids_biosscope_2021',
 '964801_v1_pump_carbohydrates_biosscope_2021',
 '964826_v1_pump_poc_pon_biosscope'}
toMerge = {'zooscan_bats_biovolume',
 'zooscan_mocness_output'}
useAsIs = {'survey_biogeochemical'};

# Start working with a data file

In [22]:
import convert #this is the function, run this cell if I have made an edit to convert.py

In [42]:
for idx in range(len(biosscope.resources)):
    data_url = biosscope.resources[idx].path
    if data_url.endswith('.csv'):
        checkFile = re.split('/',data_url).pop().replace('.csv','')
        #print(data_url)
        #have a few options and trying to group these based on added steps needed to make the data file ready
        if checkFile in toSkip:
            print('skip')
        elif checkFile in addDepth:
            print('addDepth')
        elif checkFile in toMerge:
            print('toMerge')
        elif checkFile in useAsIs:
            #print('as is')
            %run convert.py {idx}   
        else:
            print('no match')

toMerge
toMerge
Data will go here: C:\Users\klongnecker\Documents\Dropbox\GitHub_espresso\BWrepos\data
addDepth
addDepth
addDepth
addDepth


In [ ]:
#raise UserWarning('Stopping and leave code below for historical reasons, code will not run')

In [ ]:
#was heading down this path and using out as a dictionary, leave for now in case I come back to this.
# out = {};
# for idx,item in enumerate(biosscope.resources):
#     justFile = item.name;
#     #pdb.set_trace()
#     out.update({'file':justFile})
#     if justFile == '920443_v1_biosscope_in_situ_pump_chemical_data':
#         print('yes')
#         out.update({'toDo':'addDepth'})
        
#     #out.append(justFile)

# out #this will be a list, iterate through the files on this list